### optuna 튜닝


In [178]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random
from imblearn.under_sampling import RandomUnderSampler
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
plt.style.use("ggplot")
import lightgbm as lgbm


In [179]:
train_df = pd.read_csv('../Data/multi-label-classification-data/train_process.csv')
test_df = pd.read_csv('../Data/multi-label-classification-data/test.csv')
label_df = pd.read_csv('../Data/multi-label-classification-data/label.csv')
submission = pd.read_csv('../Data/multi-label-classification-data/sample_submission.csv')

In [180]:
train_df = train_df.drop(['id','FpDensityMorgan1', 'FpDensityMorgan2','FpDensityMorgan3'], axis=1)
test_df = test_df.drop(['id','FpDensityMorgan1', 'FpDensityMorgan2','FpDensityMorgan3'], axis=1)


In [181]:
# skew_feature = ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3v', 'Chi4n',
#        'EState_VSA1', 'EState_VSA2', 'ExactMolWt', 'FpDensityMorgan1',
#        'FpDensityMorgan2', 'FpDensityMorgan3',
#        'HeavyAtomMolWt', 'Kappa3', 
#        'NumHeteroatoms', 'PEOE_VSA10', 'PEOE_VSA14', 'PEOE_VSA6', 'PEOE_VSA7',
#        'PEOE_VSA8', 'SMR_VSA10', 'SMR_VSA5', 'SlogP_VSA3', 'VSA_EState9',
#        ]
skew_feature = [ 'Kappa3']

In [182]:
for col in skew_feature:
    train_df[col] = np.log1p(train_df[col])
    test_df[col] = np.log1p(test_df[col])


/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [183]:
# 훈련 검증 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(train_df, label_df,test_size=0.3)

In [184]:
cols = label_df.columns

In [195]:
# optuna 사용
import optuna
from optuna import Trial
from sklearn.metrics import roc_curve

train = lgbm.Dataset(x_train, y_train.iloc[:,0])
test = lgbm.Dataset(x_test, y_test.iloc[:,0])
def objective(trial : Trial):
    params = {
        'boosting_type' : 'gbdt',
        "n_estimators" : 10000,
#         'fold_size':trial.suggest_int('fold_size', 4, 16),
        'max_depth':trial.suggest_int('max_depth', 4, 16),
        'random_state': 722,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 8, 25),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.8, 1.0),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 8),
        'min_child_samples': trial.suggest_int('min_child_samples', 16, 64),
        'learning_rate': 0.05,
        "metric":"auc",
        #"objective":"binary",
    }
    
    model = lgbm.train(params=params, train_set=train, valid_sets=[train, test], num_boost_round=500, early_stopping_rounds=10, verbose_eval=100)
    preds = model.predict(x_test)
    print(preds)
    fpr, tpr, thresholds = roc_curve(y_test['EC1'], preds, pos_label=1)
    
    score = auc(fpr, tpr)
    
    return score
    

In [196]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
print("trial",len(study.trials))
print('best_trail', study.best_trial.params)

/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 14:02:23,629] Trial 0 finished with value: 0.700529921422621 and parameters: {'max_depth': 7, 'reg_alpha': 0.09960428243405861, 'reg_lambda': 0.22715815282277732, 'num_leaves': 22, 'colsample_bytree': 0.8722546488685065, 'subsample': 0.9395441544347632, 'subsample_freq': 5, 'min_child_samples': 48}. Best is trial 0 with value: 0.700529921422621.
/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 14:02:23,662] Trial 1 finished with value: 0.7016348625117608 and parameters: {'max_depth': 8, 'reg_alpha': 8.0091

[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of used features: 28
[LightGBM] [Info] Start training from score 0.667726
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[38]	training's auc: 0.753825	valid_1's auc: 0.70053
[0.80800271 0.50782037 0.73373739 ... 0.82173696 0.75985569 0.81448794]
[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of used features: 28
[LightGBM] [Info] Start training from score 0.667726
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[25]	training's auc: 0.729707	valid_1's auc: 0.701635
[0.78525428 0.55247812 0.73830851 ... 0.78066032 0.75125159 0.79058419]
[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of used features: 28
[LightGBM] [Info] Start training from score 0.667726
Training until validation scores don't improve for 10 rounds
Early stopping, best iterati

[I 2023-07-07 14:02:23,817] Trial 5 finished with value: 0.6969952925438707 and parameters: {'max_depth': 5, 'reg_alpha': 0.9582230982167326, 'reg_lambda': 7.152564688062882e-05, 'num_leaves': 15, 'colsample_bytree': 0.7684572786692424, 'subsample': 0.8240601373440103, 'subsample_freq': 3, 'min_child_samples': 45}. Best is trial 1 with value: 0.7016348625117608.
/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 14:02:23,884] Trial 6 finished with value: 0.6983253837160088 and parameters: {'max_depth': 12, 'reg_alpha': 0.1770251852793314, 'reg_lambda': 1.4132868948701122e-06, 'num_leaves': 25, 'colsample_bytree': 0.8034341752822411, 'subsample': 0.8616821527036511, 'subsample_freq': 6, 'min_child_samples': 18}. Best is trial 1 with value: 0.7016348625117608.
/home/bbok052

Early stopping, best iteration is:
[43]	training's auc: 0.74001	valid_1's auc: 0.696995
[0.83109623 0.59086921 0.7366271  ... 0.82702856 0.7511636  0.83109623]
[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of used features: 28
[LightGBM] [Info] Start training from score 0.667726
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[16]	training's auc: 0.736661	valid_1's auc: 0.698325
[0.7612349  0.57316165 0.73665119 ... 0.76163947 0.72925692 0.76413778]
[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of used features: 28
[LightGBM] [Info] Start training from score 0.667726
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[31]	training's auc: 0.725483	valid_1's auc: 0.698516
[0.79013955 0.55282744 0.77135012 ... 0.79172179 0.76708873 0.79263928]
[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of

/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 14:02:24,037] Trial 10 finished with value: 0.6992794287646742 and parameters: {'max_depth': 16, 'reg_alpha': 8.526243106586949e-06, 'reg_lambda': 6.199428804610007, 'num_leaves': 20, 'colsample_bytree': 0.963773366962577, 'subsample': 0.8909986376375555, 'subsample_freq': 2, 'min_child_samples': 64}. Best is trial 1 with value: 0.7016348625117608.
/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 14:02:24,076] Trial 11 finished with value: 0.6992845458120152 and parameters: {'max_depth': 4, 'reg_alpha': 3.

...

[0.73728518 0.60582024 0.73726385 ... 0.73528621 0.71575769 0.73728518]
[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of used features: 28
[LightGBM] [Info] Start training from score 0.667726
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[30]	training's auc: 0.732822	valid_1's auc: 0.700284
[0.79543013 0.59894234 0.73022464 ... 0.79664119 0.74149898 0.79543013]
[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of used features: 28
[LightGBM] [Info] Start training from score 0.667726
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[21]	training's auc: 0.72878	valid_1's auc: 0.700852
[0.76915787 0.59378231 0.75627839 ... 0.77158459 0.75826642 0.77523759]
[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of used features: 28
[LightGBM] [Info] Start training from score 0.667726
Training until va

/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 14:02:27,678] Trial 93 finished with value: 0.7006100884976281 and parameters: {'max_depth': 9, 'reg_alpha': 0.005603124986586876, 'reg_lambda': 0.0012135598046239687, 'num_leaves': 16, 'colsample_bytree': 0.7549010708685544, 'subsample': 0.9975755143697583, 'subsample_freq': 7, 'min_child_samples': 50}. Best is trial 49 with value: 0.7020565072126488.
/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 14:02:27,729] Trial 94 finished with value: 0.6992357632940321 and parameters: {'max_depth': 10, 'reg_alpha

Early stopping, best iteration is:
[24]	training's auc: 0.730126	valid_1's auc: 0.70061
[0.77689742 0.56913023 0.77125665 ... 0.78072841 0.74123761 0.78390324]
[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of used features: 28
[LightGBM] [Info] Start training from score 0.667726
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[40]	training's auc: 0.735704	valid_1's auc: 0.699236
[0.81921195 0.59883053 0.77044574 ... 0.81742558 0.74301179 0.81820367]
[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of used features: 28
[LightGBM] [Info] Start training from score 0.667726
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[36]	training's auc: 0.732368	valid_1's auc: 0.699296
[0.80131181 0.52764467 0.76159846 ... 0.8075844  0.75869998 0.8051108 ]
[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of

[I 2023-07-07 14:02:27,880] Trial 97 finished with value: 0.6990567803492647 and parameters: {'max_depth': 10, 'reg_alpha': 0.011268993521266278, 'reg_lambda': 0.010918542455586313, 'num_leaves': 21, 'colsample_bytree': 0.7082229842194017, 'subsample': 0.832033907517717, 'subsample_freq': 2, 'min_child_samples': 48}. Best is trial 49 with value: 0.7020565072126488.
/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 14:02:27,932] Trial 98 finished with value: 0.6990915762711826 and parameters: {'max_depth': 8, 'reg_alpha': 0.14576959752286883, 'reg_lambda': 0.24410014025024948, 'num_leaves': 25, 'colsample_bytree': 0.7341562469547347, 'subsample': 0.9796501900739665, 'subsample_freq': 3, 'min_child_samples': 55}. Best is trial 49 with value: 0.7020565072126488.
/home/bbok0

Early stopping, best iteration is:
[37]	training's auc: 0.749303	valid_1's auc: 0.699057
[0.81268351 0.5836007  0.77168601 ... 0.81009384 0.74522255 0.81523476]
[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of used features: 28
[LightGBM] [Info] Start training from score 0.667726
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[27]	training's auc: 0.744679	valid_1's auc: 0.699092
[0.79022886 0.59090326 0.77386765 ... 0.7881428  0.7222964  0.79605525]
[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of used features: 28
[LightGBM] [Info] Start training from score 0.667726
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[39]	training's auc: 0.741859	valid_1's auc: 0.700787
[0.80757523 0.55613591 0.7679916  ... 0.80946634 0.7304683  0.81401175]
trial 100
best_trail {'max_depth': 10, 'reg_alpha': 0.020588464488800653, 'reg_lamb

In [193]:
# 하이퍼 파라미터
# param1={"num_leaves":150,
#         "max_bin":200,
#        "feature_fraction":0.52,
#        "bagging_fraction":0.52,
#        "objective":"binary",
#        "learning_rate":0.05,
#        "boosting_type":"gbdt",
#        "metric":"auc"
#        }
# param2={"num_leaves":100,
#        "max_bin":200,
#        "feature_fraction":0.52,
#        "bagging_fraction":0.52,
#        "objective":"binary",
#        "learning_rate":0.05,
       
#        "metric":"auc"
#        }

param1 = {'max_depth': 4,
          'reg_alpha': 1.1368236970200307e-05, 
          'reg_lambda': 4.6208018285505224e-06, 
          'num_leaves': 8,
          'colsample_bytree': 0.9476860631912927, 
          'subsample': 0.8466146128214767,
          'subsample_freq': 2,
          'min_child_samples': 48,
          "metric":"auc",
          "boosting_type":"gbdt",
          "objective":"binary",
          }
param2 = {'max_depth': 6, 
          'reg_alpha': 1.6114208357018784e-08,
          'reg_lambda': 0.00013461567148605243,
          'num_leaves': 32,
          'colsample_bytree': 0.7544537836898809,
          'subsample': 0.8536948315468904,
          'subsample_freq': 6,
          'min_child_samples': 49,
          "metric":"auc",
          "boosting_type":"gbdt",
          }
param3 = {'max_depth': 15,
          'reg_alpha': 9.782691045632198e-05, 
          'reg_lambda': 0.361168894005083,
          'num_leaves': 15,
          'colsample_bytree': 0.8067711071360085,
          'subsample': 0.9660844252408497,
          'subsample_freq': 4,
          'min_child_samples': 36,
          "metric":"auc",
          "boosting_type":"gbdt",
          "objective":"binary",
          }
param4 = {'max_depth': 10,
          'reg_alpha': 0.020588464488800653,
          'reg_lambda': 0.0072620231563443795,
          'num_leaves': 17, 
          'colsample_bytree': 0.7760946649634015,
          'subsample': 0.9905813193330342,
          'subsample_freq': 3,
          'min_child_samples': 43,
          "metric":"auc",
          "boosting_type":"gbdt",}


params = [param3, param2]

In [194]:
# 학습

models = []
for  i in range(2):
    print(i)
    train = lgbm.Dataset(x_train, y_train.iloc[:,i])
    test = lgbm.Dataset(x_test, y_test.iloc[:,i])
    model = lgbm.train(params=params[i], train_set=train, valid_sets=[train, test], num_boost_round=500, early_stopping_rounds=10, verbose_eval=100)
    models.append(model)

0
[LightGBM] [Warning] Starting from the 2.1.2 version, default value for the "boost_from_average" parameter in "binary" objective is true.
This may cause significantly different results comparing to the previous versions of LightGBM.
Try to set boost_from_average=false, if your old models produce bad results
[LightGBM] [Info] Number of positive: 6935, number of negative: 3451
[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.667726 -> initscore=0.697917
[LightGBM] [Info] Start training from score 0.697917
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[26]	training's auc: 0.748048	valid_1's auc: 0.702662
1
[LightGBM] [Info] Total Bins 5552
[LightGBM] [Info] Number of data: 10386, number of used features: 28
[LightGBM] [Info] Start training from score 0.797516
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Tr

In [189]:
submission = submission[['id']]

ec1 = pd.DataFrame(models[0].predict(test_df), columns=['EC1'])
ec2 = pd.DataFrame(models[1].predict(test_df), columns=['EC2'])
submission = pd.concat([submission, ec1, ec2], axis=1)

In [190]:

submission.to_csv('submit_04_4.csv', index=False)